In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/_Git/comp0087_assignment

/content/drive/MyDrive/_Git/comp0087_assignment


In [ ]:
!pip install --quiet transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 99.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.7 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, LongT5ForConditionalGeneration
import pandas as pd
from tqdm.notebook import tqdm
from pathlib import Path

In [ ]:
# Function for summarisation
def summarise(texts, tokenizer, model, device, min_summary_legnth, max_summary_length):
    # tokenize the batch of texts
    inputs = tokenizer.batch_encode_plus(texts, return_tensors='pt', padding=True, truncation=True, max_length=512).to(device)
    
    # generate summaries for the batch of texts
    summary_ids = model.generate(inputs.input_ids, attention_mask=inputs.attention_mask, max_length=max_summary_length, min_length = min_summary_legnth)
    
    # decode the summary ids back into text
    batch_summaries = tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    
    return batch_summaries

In [ ]:
# Move to GPU
device = "cuda" if torch.cuda.is_available() else "cpu"

# Setup the model and tokenizer
model = LongT5ForConditionalGeneration.from_pretrained("google/long-t5-tglobal-base").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/long-t5-tglobal-base")

# Make folder if necessary
Path("data/echr/long_t5_summary").mkdir(parents=True, exist_ok=True)

In [ ]:
# Loop through all text in data
for set in ["train"]:
    #for name in ["non-anon", "anon"]:
    for name in ["anon"]:

        print(f"{name} {set} set")
        df = pd.read_pickle(f"data/echr/long_t5_summary/{name}_{set}.pkl")
        if "summary" not in df.columns:
            df["summary"] = ""
        i=0
        batch_size = 8
        num_batches = (len(df)//batch_size)+1

        # Loop through summarise
        for j in tqdm(range(600)):
            start_idx = j*batch_size
            end_idx = (j+1)*batch_size
            texts = df['text'][start_idx:end_idx].tolist()
            batch_summaries = summarise(texts, tokenizer, model, device, min(256, min(df['text'][start_idx:end_idx].str.split().apply(len))), 512)

            for k, summary in enumerate(batch_summaries):
                df["summary"][start_idx+k] = summary

            # Save every 50 summaries in case of crashes
            if (j % 50==0) and (j>0):
                df.to_pickle(f"data/echr/long_t5_summary/{name}_{set}.pkl")
            
        # Save summary
        df.to_pickle(f"data/echr/long_t5_summary/{name}_{set}.pkl")

anon train set


  0%|          | 0/600 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/modeling_utils.py:764: FutureWarning: The `device` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(
<ipython-input-7-925ec3f01309>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["summary"][start_idx+k] = summary
